In [ ]:
from docx import *
from string import ascii_uppercase
from docx.enum.text import WD_COLOR_INDEX
import pandas as pd
import re

In [ ]:
def replaceData (x) :
    for idx in range(len(x)):
        x[idx] = re.sub("¤","fi",x[idx])
        x[idx] = re.sub("¶","fl",x[idx])
        x[idx] = re.sub("\s-\s","-",x[idx])
        x[idx] = re.sub("-\s","",x[idx])
        x[idx] = re.sub("\s{2}|\s{3}"," ",x[idx])
        x[idx] = re.sub("\s*$|\xad|\xad\s*","",x[idx])
        x[idx] = re.sub("^\)|^\)\s*","",x[idx])
        x[idx] = re.sub("\(\s\)|\(\s=\)|\(=\)|\(=$|\($|\=$","",x[idx])
        x[idx] = re.sub("^\.\s\w\)\s|^\w\)\s*|\s\w\)$","",x[idx])
        x[idx] = re.sub("^\s*|^\s|^\s\w","",x[idx])
        x[idx] = re.sub("\[and","",x[idx])
        x[idx] = re.sub("\[|\[\s","",x[idx])
        x[idx] = re.sub("\]","",x[idx])
        x[idx] = re.sub("\s-\s|-\s|\s-","-",x[idx])
        x[idx] = re.sub("\s<\s|\s<|<\s|<","",x[idx])
        x[idx] = re.sub("^\s","",x[idx])
        x[idx] = re.sub("\s\)",")",x[idx])
        x[idx] = re.sub("\(\s","(",x[idx])
        x[idx] = re.sub("^/.*/","",x[idx])
        x[idx] = re.sub("\;$","",x[idx])
        x[idx] = re.sub("\:$","",x[idx]) #sekar
        x[idx] = re.sub("\s/\s|\s/|/\s","/",x[idx])
        x[idx] = re.sub("\(or\,$","",x[idx])
        x[idx] = re.sub("\snya$","nya",x[idx])
        x[idx] = re.sub("\snya\s","nya ",x[idx])
        x[idx] = re.sub("\skan$","kan",x[idx])
        x[idx] = re.sub("\skan\s","kan ",x[idx])
        
        x[idx] = re.sub("\s\?","?",x[idx])
        x[idx] = re.sub("\.$|\s\.$|\.\s$","",x[idx])
        x[idx] = re.sub("\.\.$","...",x[idx])
        x[idx] = re.sub("\s\.\s",". ",x[idx])
        x[idx] = re.sub("\s\,\s",", ",x[idx])
        
        x[idx] = re.sub("someone ’s","someone’s",x[idx])
        x[idx] = re.sub("ke pada","kepada",x[idx])
        x[idx] = re.sub("^se ","se",x[idx])
        x[idx] = re.sub("^ber ","ber",x[idx])
        x[idx] = re.sub("\s-kah$","kah",x[idx])
        x[idx] = re.sub("\s-kah\s","kah ",x[idx])
        
        x[idx] = re.sub("\sesp\s|\sesp|esp\s"," especially ",x[idx])
        x[idx] = re.sub("\(usu\s","(usually ",x[idx])
        x[idx] = re.sub("\susu\s"," usually ",x[idx])
        
        
        x[idx] = re.sub("_","...",x[idx])
        x[idx] = re.sub("\.\.$","...",x[idx])
    
    return x

# fungsi untuk menyimpan hasil ke bentuk list
def cleanData(data):
    x = replaceData(data)
    
    #split similar
    if x[4] != "" and re.search(r'\(\=\s(.*?)\)',x[4]):
        x[4] = re.search(r'\(\=\s(.*?)\)',x[4]).group(1)
        x[4] = re.split(",",x[4])
    
    #split panah
    if x[5] != "" :        
        x[5] = re.split(",",x[5])
        listPanah = []
        
        for idx in range(len(x[5])):
            x[5][idx] = re.sub("^\s","",x[5][idx])
            
            if '(' in x[5][idx] :
                x[5][idx], y = re.sub(r"[/(].*?[/))=]","",x[5][idx]),re.sub(r"[()=]","",x[5][idx])
                x[5][idx], y = re.sub(r"\s$","",x[5][idx]),re.sub(r"\s$","",y)
                x[5][idx] = re.sub("\s{2}|\s{3}"," ",x[5][idx])
                listPanah.append(y)
        
        if len(listPanah) != 0 :
            for val in listPanah :
                x[5].append(val)
                
                
    # split perbandingan
    if x[8] != "" :
        x[8] = re.split(",",x[8])
        
        for idx in range(len(x[8])):
            x[8][idx] = re.sub("^\s","",x[8][idx])
        
    
    #split terjemahan
    if x[11] != "" :
        x[11] = re.split(",",x[11])
        x[11] = replaceData(x[11])
        
        if "" in x[11] :
            x[11].remove("")
        
        # memindah data panah ke akronim
        if "(in acronyms)" in x[11] :
            x[6] = x[5]
            x[5] = ""
        
        for idx in range(len(x[11])):
            x[11][idx] = re.sub("e\*g\*","e.g.",x[11][idx])
            x[11][idx] = re.sub("i\*e\*","i.e.",x[11][idx])
            x[11][idx] = re.sub("\*etc",", etc",x[11][idx])
            x[11][idx] = re.sub("etc \.","etc.",x[11][idx])
            x[11][idx] = re.sub("\.$|\s\.$|\.\s$","",x[11][idx])
            x[11][idx] = re.sub("\(in acronyms\)","",x[11][idx])
            x[11][idx] = re.sub("^\.\s|^\.","",x[11][idx])
            
            if(re.search("\d\s\*\s\d", x[11][idx])) :
                x[11][idx] = re.sub("\s\*\s|\s\*|\*\s|\*",",",x[11][idx])
            else : 
                x[11][idx] = re.sub("\*|\s\*|\*\s|\s\*\s",",",x[11][idx])
            
        if len(x[11]) < 1 :
            x[11] = ""
        elif len(x[11]) == 1 and x[11][0] == "" :
            x[11] = ""
    
    y = pd.DataFrame(data=[x],columns=["lema","sublema","angka romawi","gabungan","similar","panah","akronim","antonim","perbandingan","keterangan","angka arab","terjemahan"])
    
    return x,y

def checkListNull(data) :
    isNull = True
    
    for item in data:
        if item != "" :
            isNull = False
    
    return isNull

def cleanParenthesis(word) :
    if '(' in word :

        if (re.search("\(=", word) ) :
            x, y = re.sub(r"[\(=].*?[\)]","",word),re.sub(r"[(=)]","",word)
        else :
            x, y = re.sub(r"[\(].*?[\)]","",word),re.sub(r"[()]","",word)

        
        x, y = re.sub(r"\s$","",x),re.sub(r"\s$","",y)
        word = x
        similar = y
        
    else :
        word = re.sub(r"[()]|\s$","",word)
        similar = ""
        
    results = {
        "word" : word,
        "similar" : similar
    }
        
    return results

def wordBold(tipe, text, tempData) :
    
    lema = ""
    sublema = ""
    angkaRomawi = ""
    angkaArab = ""
    similar = ""
    
    # SIMILAR (AND)
    # jika kata sebelum kata style bold adalah "and" maka dianggap sebagai similar
#     if (index > 0 and paragraph.runs[index-1].text in listAnd) or (index > 1 and paragraph.runs[index-2].text in listAnd) or (index > 0 and re.search(",", paragraph.runs[index-1].text)) or (index > 1 and re.search(",", paragraph.runs[index-2].text)) or (index > 0 and re.search("\=", paragraph.runs[index-1].text)) or (index > 1 and re.search("\=", paragraph.runs[index-2].text)) :
    if index > 0 and ((prevText in listAnd) or (re.search(",", prevText)) or (re.search("\=", prevText))) and not (re.match("\d", text) or text in listRomanNumeral) and not re.search("\.", prevText): 
#         similar = text
        
        if re.search("\s", text) :
            data = text.split()
            
            for item in data :
                if item in listRomanNumeral:
                    angkaRomawi = item

                elif len(item) < 3 and re.match("\d", item) and not re.search("[A-Za-z]", item)  :
                    angkaArab = item

                else :
                    if similar == "" :
                        similar = item
                    else :
                        similar += " "+item 
        else :
            similar = text

    else :
        if re.search("\s", text) :
            words = text.split()

            for word in words :
                if word in listRomanNumeral:
                    angkaRomawi = word 

                elif len(word) < 3 and re.match("\d", word) and not re.search("[A-Za-z]", word) :
                    angkaArab = word

                else :
                    if angkaRomawi != "" :
                        tipe = "sublema"
                        
                    if tipe == "lema" :
                        if lema == "" :
                            lema = word
                        
                        else :
                            lema += " "+word
                        
                    else :
                        if sublema == "" :
                            sublema = word
                        
                        else :
                            sublema += " "+word

        else :
            if text in listRomanNumeral and index > 0:
                angkaRomawi = text

            elif len(run.text) < 3 and re.match("\d", text) and not re.search("[A-Za-z]", text) :
                angkaArab = text

            else :
                if (tipe == "lema") :
                    lema = text
                else :
                    sublema = text

        if (tipe == "lema") :
            resultsCleanParenthesis = cleanParenthesis(lema)
            lema = resultsCleanParenthesis["word"]
            similar = resultsCleanParenthesis["similar"]
        else :
            resultsCleanParenthesis = cleanParenthesis(sublema)
            sublema = resultsCleanParenthesis["word"]
            similar = resultsCleanParenthesis["similar"]
        
                
    results = {
        "lema" : lema,
        "sublema" : sublema,
        "angkaRomawi" : angkaRomawi,
        "similar" : similar,
        "angkaArab" : angkaArab
    }

    return results

def wordItalic(text,index,paragraph,gabungan,keterangan,terjemahan) :     
    if text == "opp" or text in ["cp", "cp ", " cp", " cp "]:
        text = ""

    #KETERANGAN
    # mengambil keterangan
    if (re.search("\(", prevText) or keterangan != "") and re.search("–|~", text) == None :

        if re.search("\s", text) :
            words = text.split()

            for word in words :
                if word in listKet or re.sub("\?","",word) in listKet :
                    if keterangan == "" :
                        keterangan = word
                    else :
                        keterangan += " "+word
                else : 
                    if gabungan == "" :
                        gabungan = word
                    else :
                        gabungan += " "+word

        elif text in listKet or re.sub("\?","",text) in listKet :
            if keterangan == "" :
                keterangan = text
            else :
                keterangan += " "+text

        else :
            if (index+1 < len(paragraph.runs) and paragraph.runs[index+1].italic) or (index+1 < len(paragraph.runs) and re.search("\)", paragraph.runs[index+1].text)) :
                if gabungan == "" :
                    gabungan = text
                else :
                    gabungan += " "+text

            else :
                if text not in listRomanNumeral and isPanah != True and isPerbandingan != True :
                    if terjemahan == "" :
                        terjemahan = text
                    else :
                        terjemahan += " "+text
                
    # mengambil keterangan selanjutnya jika terdiri dari > 1 keterangan
#     elif keterangan != "" and text not in keterangan:
#         if re.search("\s", text) :
#             words = text.split()

#             for word in words :
#                 if word in listKet or re.sub("\?","",word) in listKet :
#                     if keterangan == "" :
#                         keterangan = word
#                     else :
#                         keterangan += " "+word

#         elif text in listKet or re.sub("\?","",text) in listKet :
#             if keterangan == "" :
#                 keterangan = text
#             else :
#                 keterangan += " "+text
                
#         else :
#             if text not in listRomanNumeral and panah != True and perbandingan != True :
#                 if terjemahan == "" :
#                     terjemahan = text
#                 else :
#                     terjemahan += " "+text
                
    else :
        if gabungan == "" :
            gabungan = text
        else :
            gabungan += " "+text

    return gabungan,keterangan,terjemahan

In [ ]:
# listDoc = ['551-605_New Raw']
# listDoc = ['U-Prep']
# listDoc = ["testing"]
# listDoc = ["data-prep-testing"]
# listDoc = ["1-55"]
listDoc = ["1-440~[Prep]-v2"]

for doc in listDoc : 
    # put file document
#     document = Document("New Raw/"+doc+".docx")
#     document = Document("Preprocessing/Preprocessing Perhalaman/"+doc+".docx")
#     document = Document(doc+".docx")
    document = Document("Fix Raw/"+doc+".docx")
    
    lexicon = pd.DataFrame(columns=["lema","sublema","angka romawi","gabungan","similar","panah","akronim","antonim","perbandingan","keterangan","angka arab","terjemahan"])
    
    # Hapus dari list jika huruf yang dieksekusi == huruf romawi
    listRomanNumeral = ["I","II","III","IV","V","VI","VII","VIII","IX","X","XI","XII","XIII","XIV","XV","XVI","XVII","XVIII","XIX","XX"]
    listKet = ["also Balinese","?","A","abbr","Ac","acr","adj","AE","anat","app","Bal","Ban","Bat","BD","BE","BG","bio","biol","bot","C","chem","cla","col","coq","cp","D","D/E","derog","E","elec","e.o.","epist","euph","exclam","fin","Fr","G","geo","geol","Gr","gram","Hind","IBT","infr","init","insur","Irja","Isl","J","J/Jv","joc","Jp","Jv","K","k.o.","L","leg","ling","lit","M","Mad","Mal","math","Med","med","mil","Min","mod","mus","naut","ob","obj","O jv","onom","opp","o.s.","Pal","Pap","Pers","petro","phys","pl","pl obj","pl subj","poet","Port","Pr","pron","q.v.","RC","reg","rel","rev","S","sg","Sg","Skr","Skr neo","sl","s.o.","s.o.'s","s.t.","stat","subj","Sum","Tag","Tam","voc","vulg","zod","neo"]
    listData = ["something", "someone", "oneself", "each other","something ", "someone ", "oneself ", "each other "]
    listAnd = ["and", " and", "and ", " and ", "[and", "[and ",") and",") and ","(and ","(and"]
                 
    # variabel
    lema = ""
    sublema = ""
    angkaRomawi = ""  
    gabungan = ""
    similar = ""
    panah = ""
    akronim = ""
    antonim = ""
    perbandingan = ""
    keterangan = ""
    angkaArab = ""
    terjemahan = ""
    
    prevText = ""
    prevTextStyle = ""
    
    isAkronim = False
    isSimilar = False
    isPanah = False
    isPerbandingan = False
    isBoldItalic = False
    
    tempData = {
        "lema" : "",
        "sublema" : "",
        "gabungan" : "",
    }    
    
    
#     index = 0
    
    # looping paragraf tiap document
    for paragraph in document.paragraphs:
        
        boolLemaSublema = False

        index = 0
#         prevText = ""

        # menyimpan nilai indentasi (first indent dan left indent)
        first = paragraph.paragraph_format.first_line_indent
        left = paragraph.paragraph_format.left_indent
        
#         print(paragraph.text)
#         print(len(paragraph.runs))
        
        # looping setiap kata di setiap paragrafnya
        for run in paragraph.runs:

            # mengatasi left indent == none, dengan mengubah nilai variabel left dengan tempLeft (nilai pada left sebelumnya)
            if left == None :
                left = 228600
            
            if run.text.isspace() == False :
#                 print("text : "+run.text)
#                 print(index)
#                 print(len(paragraph.runs))
#                 print(tempData)

                # deteksi kata bold
                if run.bold and run.italic == None  :
#                     print("bold :"+run.text)
#                     if (terjemahan != "") :
#                         terjemahan += " "+run.text
#                         prevText = run.text 
#                         index += 1
#                         continue
                    
                    # baca indent lema
                    if (((first == None or first == 0 or first == 76200 or first == -635) and (left <= 76835)) or ((first == -152400 or first == -153035) and (left >= 227965 or left <= 76835))) :
                        
#                         print(run.text)
                        
                        if (re.search("\-$|\<$", run.text) or re.search("\-$|\<$", lema)) and lema != "" :
                            lema = lema+run.text
                            
                            if re.search("\(*?\)",lema):
                                resultsCleanParenthesis = cleanParenthesis(lema)
                                lema = resultsCleanParenthesis["word"]
                                similar = resultsCleanParenthesis["similar"]
                                
                            if lema != "" :
                                tempData["lema"] = lema
                            if sublema != "" :
                                tempData["sublema"] = sublema
                            
                            prevText = run.text 
                            index += 1
                            continue
                        elif (re.search("\-$|\<$", run.text) or re.search("\-$|\<$", sublema)) and sublema != "" :
                            sublema = sublema+run.text
                            
                            if re.search("\(*?\)",sublema):
                                resultsCleanParenthesis = cleanParenthesis(sublema)
                                sublema = resultsCleanParenthesis["word"]
                                similar = resultsCleanParenthesis["similar"]
                                
                            if sublema != "" :
                                tempData["sublema"] = sublema
                            
                            prevText = run.text 
                            index += 1
                            continue
                        elif (re.search("\-$|\<$", run.text) or re.search("\-$|\<$", similar)) and similar != "" :
                            similar = similar+run.text
                            prevText = run.text 
                            index += 1
                            continue
                        
                        tempLeft = left
                        
                        # list yang akan dimasukkan ke lexicon
                        
                        if run.text not in listRomanNumeral and run.bold and run.italic == None :
                            listDataLexicon = [lema,sublema,angkaRomawi,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,angkaArab,terjemahan]

                            if checkListNull(listDataLexicon) == False :
                                # insert lexicon

                                x = cleanData(listDataLexicon)

                                if checkListNull(x[0]) == False :
                                    print(x[0])

                                    lexicon  = lexicon.append([x[1]],ignore_index=True)

                                    # clear data
                                    lema = ""
                                    sublema = ""
                                    angkaRomawi = ""  
                                    gabungan = ""
                                    similar = ""
                                    panah = ""
                                    akronim = ""
                                    antonim = ""
                                    perbandingan = ""
                                    keterangan = ""
                                    angkaArab = ""
                                    terjemahan = ""
                                    
                                    isAkronim = False
                                    isSimilar = False
                                    isPanah = False
                                    isPerbandingan = False
                                    isBoldItalic = False
                                
#                                 tempData = {} 
                        
                        # jika lema kosong
                        if lema == "" and index < 1 :
                            dataWordBold = wordBold("lema", run.text, tempData)

                            lema = dataWordBold["lema"]
                            sublema = dataWordBold["sublema"]
                            angkaRomawi = dataWordBold["angkaRomawi"]
                            similar = dataWordBold["similar"]
                            angkaArab = dataWordBold["angkaArab"]

                            if lema != "" :
                                tempData["lema"] = lema
                                
                            if sublema != "" :
                                tempData["sublema"] = sublema
                        
                        # jika lema tidak kosong
                        else :
                            
                            if run.text not in listRomanNumeral and run.bold and run.italic == None :
                                # list yang akan dimasukkan ke lexicon
                                listDataLexicon = [lema,sublema,angkaRomawi,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,angkaArab,terjemahan]

                                if checkListNull(listDataLexicon) == False :
                                    # insert lexicon
                                    x = cleanData(listDataLexicon)

                                    if checkListNull(x[0]) == False :
                                        print(x[0])
                                        lexicon  = lexicon.append([x[1]],ignore_index=True)

                                        # clear data
                                        lema = ""
                                        sublema = ""
                                        angkaRomawi = ""  
                                        gabungan = ""
                                        similar = ""
                                        panah = ""
                                        akronim = ""
                                        antonim = ""
                                        perbandingan = ""
                                        keterangan = ""
                                        angkaArab = ""
                                        terjemahan = ""
                                        
                                        isAkronim = False
                                        isSimilar = False
                                        isPanah = False
                                        isPerbandingan = False
                                        isBoldItalic = False

    #                                     tempData = {} 
                                        
                            dataWordBold = wordBold("sublema", run.text, tempData)

                            sublema = dataWordBold["sublema"]
                            angkaRomawi = dataWordBold["angkaRomawi"]
                            similar = dataWordBold["similar"]
                            angkaArab = dataWordBold["angkaArab"]
                            
                            if lema != "" :
                                tempData["lema"] = lema
                            if sublema != "" :
                                tempData["sublema"] = sublema
                            
                    # baca indent sublema
                    else :
 
                        if (re.search("\-$|\<$", run.text) or re.search("\-$|\<$", sublema)) and sublema != "" :
                            sublema = sublema+run.text
                            
                            if re.search("\(*?\)",sublema):
                                resultsCleanParenthesis = cleanParenthesis(sublema)
                                sublema = resultsCleanParenthesis["word"]
                                similar = resultsCleanParenthesis["similar"]
                                
                            if sublema != "" :
                                tempData["sublema"] = sublema
                              
                            prevText = run.text 
                            index += 1
                            continue
                        elif (re.search("\-$|\<$", run.text) or re.search("\-$|\<$", similar)) and similar != "" :
                            similar = similar+run.text
                            prevText = run.text 
                            index += 1
                            continue
                            
                        # list yang akan dimasukkan ke lexicon
                        listDataLexicon = [lema,sublema,angkaRomawi,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,angkaArab,terjemahan]
                        
                        if checkListNull(listDataLexicon) == False :
                            # insert lexicon
                            x = cleanData(listDataLexicon)
    
                            if checkListNull(x[0]) == False :
                                print(x[0])
                                lexicon  = lexicon.append([x[1]],ignore_index=True)
                    
                                # clear data
                                lema = ""
                                sublema = ""
                                angkaRomawi = ""  
                                gabungan = ""
                                similar = ""
                                panah = ""
                                akronim = ""
                                antonim = ""
                                perbandingan = ""
                                keterangan = ""
                                angkaArab = ""
                                terjemahan = ""
                                
                                isAkronim = False
                                isSimilar = False
                                isPanah = False
                                isPerbandingan = False
                                isBoldItalic = False

                        
                        # jika sublema kosong
                        if sublema == "" :
                            dataWordBold = wordBold("sublema", run.text, tempData)

                            sublema = dataWordBold["sublema"]
                            angkaRomawi = dataWordBold["angkaRomawi"]
                            similar = dataWordBold["similar"]
                            angkaArab = dataWordBold["angkaArab"]
                            
#                             print(dataWordBold)

                            if sublema != "" :
                                tempData["sublema"] = sublema
                        
                        # jika sublema tidak kosong
                        else :
                            # list yang akan dimasukkan ke lexicon
                            listDataLexicon = [lema,sublema,angkaRomawi,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,angkaArab,terjemahan]

                            if checkListNull(listDataLexicon) == False :
                                # insert lexicon
                                x = cleanData(listDataLexicon)

                                if checkListNull(x[0]) == False :
                                    print(x[0])
                                    lexicon  = lexicon.append([x[1]],ignore_index=True)

                                    # clear data
                                    lema = ""
                                    sublema = ""
                                    angkaRomawi = ""  
                                    gabungan = ""
                                    similar = ""
                                    panah = ""
                                    akronim = ""
                                    antonim = ""
                                    perbandingan = ""
                                    keterangan = ""
                                    angkaArab = ""
                                    terjemahan = ""
                                    
                                    isAkronim = False
                                    isSimilar = False
                                    isPanah = False
                                    isPerbandingan = False
                                    isBoldItalic = False
                                    
#                                     tempData = {} 
                                        
                            dataWordBold = wordBold("sublema", run.text, tempData)

                            sublema = dataWordBold["sublema"]
                            angkaRomawi = dataWordBold["angkaRomawi"]
                            similar = dataWordBold["similar"]
                            angkaArab = dataWordBold["angkaArab"]
                        
#                             print(dataWordBold)
                            
                            if sublema != "" :
                                tempData["sublema"] = sublema
                
                    if similar != "" or angkaRomawi != "" :
                        resultsCleanParenthesis = cleanParenthesis(similar)

                        for key,value in resultsCleanParenthesis.items() :
                            if value != "" :
                                similar = value
                                
                                # list yang akan dimasukkan ke lexicon
                                listDataLexicon = [lema,sublema,angkaRomawi,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,angkaArab,terjemahan]

                                if checkListNull(listDataLexicon) == False :
                                    # insert lexicon
                                    x = cleanData(listDataLexicon)

                                    if checkListNull(x[0]) == False :
                                        print(x[0])
                                        lexicon  = lexicon.append([x[1]],ignore_index=True)

                                        # clear data
                                        lema = ""
                                        sublema = ""
                                        angkaRomawi = ""  
                                        gabungan = ""
                                        similar = ""
                                        panah = ""
                                        akronim = ""
                                        antonim = ""
                                        perbandingan = ""
                                        keterangan = ""
                                        angkaArab = ""
                                        terjemahan = ""
                                        
                                        isAkronim = False
                                        isSimilar = False
                                        isPanah = False
                                        isPerbandingan = False
                                        isBoldItalic = False
                                
                    prevTextStyle = "bold"
                            
                # deteksi kata italic (kata gabungan, keterangan)
                elif (run.italic and run.bold) or run.italic :
#                     print("italic :"+run.text)
                    
                    if run.italic and run.bold :
                        isBoldItalic = True
                    
                    if ((prevText in listAnd) or re.search("^=", prevText) or (re.search("^\(=", prevText) and re.search("\)$", paragraph.runs[index+1].text)) or re.search('\(\=\s(.*?)\)', run.text)) :
                        isSimilar = True
                
                    if isSimilar == True and similar != "" :
                        similar += " "+run.text
                        
#                     if (list(filter(lambda x : x == True, list(map(lambda x : prevText.endswith(x), listData))))) or ((list(filter(lambda x : x == True, list(map(lambda x : terjemahan.endswith(x), listData)))))) or re.search("_|;",prevText) :
                    if (list(filter(lambda x : x == True, list(map(lambda x : prevText.endswith(x), listData))))) or ((list(filter(lambda x : x == True, list(map(lambda x : terjemahan.endswith(x), listData)))))) or (re.search("_|;",run.text) and terjemahan != "") :
                        # list yang akan dimasukkan ke lexicon
                        listDataLexicon = [lema,sublema,angkaRomawi,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,angkaArab,terjemahan]

                        if checkListNull(listDataLexicon) == False :
                            # insert lexicon
                            x = cleanData(listDataLexicon)

                            if checkListNull(x[0]) == False :
                                print(x[0])
                                lexicon  = lexicon.append([x[1]],ignore_index=True)

                                # clear data
                                lema = ""
                                sublema = ""
                                angkaRomawi = ""  
                                gabungan = ""
                                similar = ""
                                panah = ""
                                akronim = ""
                                antonim = ""
                                perbandingan = ""
                                keterangan = ""
                                angkaArab = ""
                                terjemahan = ""
                                
                                isAkronim = False
                                isSimilar = False
                                isPanah = False
                                isPerbandingan = False
                                isBoldItalic = False
                                
                            dataWordItalic = wordItalic(run.text,index,paragraph,gabungan,keterangan,terjemahan)
                            
                            gabungan = dataWordItalic[0]
                            keterangan = dataWordItalic[1]
                            terjemahan = dataWordItalic[2]
                            
                            tempData["gabungan"] = ""
                            tempData["gabungan"] = gabungan.split()
                                
                    elif antonim != "" :
                        antonim = antonim+" "+run.text
                        
                    elif isPanah == False and re.search(">", prevText) :
                        panah = re.sub("\.","", run.text)
                        isPanah = True
                        run.text = ""
                        
                    elif isPerbandingan == False and re.search("cp", prevText) :
                        perbandingan = re.sub("\.","", run.text)
                        isPerbandingan = True
                        run.text = ""
                        
                    else :
                        dataWordItalic = wordItalic(run.text,index,paragraph,gabungan,keterangan,terjemahan)

                        gabungan = dataWordItalic[0]
                        keterangan = dataWordItalic[1]
                        terjemahan = dataWordItalic[2]
                        
                        tempData["gabungan"] = ""
                        tempData["gabungan"] = gabungan.split()
                        
#                         print(dataWordItalic)

                    prevTextStyle = "italic"

                # deteksi kata reguler (akronim, terjemahan)
                else :
#                     print("regular : "+run.text)
                    
                    if re.search("–", gabungan) or re.search("~", gabungan) :

                        if (isSimilar == False) :
                            if re.search("–", gabungan) :
                                gabungan = re.sub("–",tempData["lema"],gabungan)
                                gabungan = re.sub("  "," ", gabungan)

                            if re.search("~", gabungan) :
                                gabungan = re.sub("~",tempData["sublema"],gabungan)
                                gabungan = re.sub("  "," ", gabungan)

                            resultsCleanParenthesis = cleanParenthesis(gabungan)

                            if (similar == "") or re.search("\(", gabungan) :
                                gabungan = resultsCleanParenthesis["word"]
                                similar = resultsCleanParenthesis["similar"]
                            else :
                                gabungan = ""
                                similar = resultsCleanParenthesis["similar"]
                        else :
                            
                            if re.search("–", gabungan) :
                                similar = re.sub("–",tempData["lema"],gabungan)
                                similar = re.sub("  "," ", similar)

                            if re.search("~", gabungan) :
                                similar = re.sub("~",tempData["sublema"],gabungan)
                                similar = re.sub("  "," ", similar)
                                
                            gabungan = ""

                            resultsCleanParenthesis = cleanParenthesis(similar)

                            for key,value in resultsCleanParenthesis.items() :
                                if value != "" :
                                    similar = value

                                    # list yang akan dimasukkan ke lexicon
                                    listDataLexicon = [lema,sublema,angkaRomawi,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,angkaArab,terjemahan]

                                    if checkListNull(listDataLexicon) == False :
                                        # insert lexicon
                                        x = cleanData(listDataLexicon)

                                        if checkListNull(x[0]) == False :
                                            print(x[0])
                                            lexicon  = lexicon.append([x[1]],ignore_index=True)

                                            # clear data
                                            lema = ""
                                            sublema = ""
                                            angkaRomawi = ""  
                                            gabungan = ""
                                            similar = ""
                                            panah = ""
                                            akronim = ""
                                            antonim = ""
                                            perbandingan = ""
                                            keterangan = ""
                                            angkaArab = ""
                                            terjemahan = ""

                                            isAkronim = False
                                            isSimilar = False
                                            isPanah = False
                                            isPerbandingan = False
                                            isBoldItalic = False
                            
                        # list yang akan dimasukkan ke lexicon
                        listDataLexicon = [lema,sublema,angkaRomawi,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,angkaArab,terjemahan]

                        if checkListNull(listDataLexicon) == False :
                            # insert lexicon
                            x = cleanData(listDataLexicon)

                            if checkListNull(x[0]) == False :
                                print(x[0])
                                lexicon  = lexicon.append([x[1]],ignore_index=True)

                                # clear data
                                lema = ""
                                sublema = ""
                                angkaRomawi = ""  
                                gabungan = ""
                                similar = ""
                                panah = ""
                                akronim = ""
                                antonim = ""
                                perbandingan = ""
                                keterangan = ""
                                angkaArab = ""
                                terjemahan = ""
                                
                                isAkronim = False
                                isSimilar = False
                                isPanah = False
                                isPerbandingan = False
                                isBoldItalic = False

                    else :
                        word = run.text

                        if gabungan != "" :

                            if isPanah == True and panah != run.text and run.text not in listRomanNumeral :
                                panah += " "+gabungan
                                gabungan = ""
                                tempData["gabungan"] = ""
                                
                            elif isPerbandingan == True and perbandingan != run.text and run.text not in listRomanNumeral :
                                perbandingan += " "+gabungan
                                gabungan = ""
                                tempData["gabungan"] = ""
                                
                            elif isSimilar == True and panah != run.text and run.text not in listRomanNumeral :
                                similar += " "+gabungan
                                gabungan = ""
                                tempData["gabungan"] = ""
                                
                            elif isBoldItalic == True :
                                                                
                                resultsCleanParenthesis = cleanParenthesis(gabungan)

                                if (isSimilar != True) or re.search("\(", gabungan) :
                                    gabungan = resultsCleanParenthesis["word"]
                                    similar = resultsCleanParenthesis["similar"]
                                else :
#                                     gabungan = resultsCleanParenthesis["word"]
                                    similar = resultsCleanParenthesis["similar"]
                                
                                isBoldItalic = False
                                isSimilar = False
                                tempData["gabungan"] = ""

#                                 print(gabungan)
#                                 print(similar)
                                
                            elif len(tempData["gabungan"]) > 1 and ((tempData['lema'] != "" and tempData['lema'].lower() in gabungan.lower()) or ( tempData['sublema'] != "" and tempData['sublema'].lower() in gabungan.lower())) :
                                resultsCleanParenthesis = cleanParenthesis(gabungan)
#                                 print("gabungan : "+gabungan)
                        
                                if (isSimilar != True) or re.search("\(", gabungan) :
                                    gabungan = resultsCleanParenthesis["word"]
                                    similar = resultsCleanParenthesis["similar"]
                                else :
#                                     gabungan = resultsCleanParenthesis["word"]
                                    similar = resultsCleanParenthesis["similar"]
                                
                                isSimilar = False
                                tempData["gabungan"] = ""
                                    
                            elif (index > 1 and (len(tempData["gabungan"]) > 0 and len(tempData["gabungan"]) <= 3)) or (index == len(paragraph.runs)-1 and len(tempData["gabungan"]) > 1) :
                                if (isSimilar != True) :
                                    terjemahan = terjemahan+" "+gabungan
                                else :
                                    similar = gabungan

                                isSimilar = False
                                tempData["gabungan"] = ""
                                gabungan = ""
                                
                            else :
                                resultsCleanParenthesis = cleanParenthesis(gabungan)
                        
                                if similar == "" :
                                    if (isSimilar != True) or re.search("\(", gabungan) :
                                        gabungan = resultsCleanParenthesis["word"]
                                        similar = resultsCleanParenthesis["similar"]
                                    else :
                                        gabungan = ""
                                        similar = resultsCleanParenthesis["similar"]
                                    
                                isSimilar = False
                                tempData["gabungan"] = ""

                    if re.search("\[", prevText) :
                        isAkronim = True
                        tempData["akronim"] = ""
                        tempData["akronim"] = run.text 
                        
                        if (index+1 < len(paragraph.runs) and re.search("\]", paragraph.runs[index+1].text)) or re.search("\]", run.text) :
                            akronim = tempData["akronim"]
                            isAkronim = False
                            
                    elif isAkronim == True and run.text not in akronim :
                        tempData["akronim"] += " "+run.text 
                        
                        if (index+1 < len(paragraph.runs) and re.search("\]", paragraph.runs[index+1].text)) or re.search("\]", run.text) :
                            akronim = tempData["akronim"]
                            isAkronim = False
                            
#                         print("akronim : "+akronim)
                    
                    else :
                        
                        if isPanah == True and re.search("\.|;", run.text) :
                            
#                             panah = re.sub("\.|\s\.","",run.text)
                            panah += " "+run.text
                            isPanah = False
                            run.text = ""
                        
                        if isPanah == True and run.text not in listRomanNumeral :
                            text = ""
                            text = re.sub("\.","", run.text)
                            
                            panah += " "+text
                            run.text = ""

                        if isPerbandingan == True and re.search("\.|;", run.text) :
#                             perbandingan = re.sub("\.|\s\.","",perbandingan)
                            perbandingan += " "+run.text
                            isPerbandingan = False
                            run.text = ""
                            
                        if isPerbandingan == True and run.text not in listRomanNumeral :
                            text = ""
                            text = re.sub("\.","", run.text)
                            
                            perbandingan += " "+text
                            run.text = ""
                            
                        if run.text != "." and re.search(">", prevText) :
                            panah = re.sub("\.","", run.text)
                            isPanah = True
                            run.text = ""

                        elif run.text != "." and prevText in ["cp", "cp ", " cp", " cp "]  :
                            perbandingan = run.text
                            isPerbandingan = True
                            run.text = ""
                        
                        elif antonim == "" and prevTextStyle == "italic" and prevText == "opp":
                            antonim = run.text
                        
                        else :

#                             print(run.text)
                            #TERJEMAHAN
                            if run.text in listAnd and (index == 0 or (index > 0 and (prevTextStyle == "bold" or prevTextStyle == "italic" or re.search("\]|\)|\(", prevText)) or (index > 1 and re.search("\]|\)|\(", prevText)))) and ((index+1 < len(paragraph.runs) and (paragraph.runs[index+1].bold == True or paragraph.runs[index+1].italic == True)) or (index+2 < len(paragraph.runs) and (paragraph.runs[index+2].bold == True or paragraph.runs[index+2].italic == True))):
                                prevText = run.text
                                index += 1
                                continue

                            if run.text == "[" or run.text == "]" or run.text in [" > "," >","> ",">"] :
                                prevText = run.text 
                                index += 1
                                continue
                                
                            if re.search("\]\s", run.text) :
                                run.text = re.sub("\]\s","",run.text)

                            if isPanah == False and isPerbandingan == False :
                                if terjemahan == "" :
                                    terjemahan = run.text

                                elif terjemahan != "" :
                                    terjemahan = terjemahan+" "+run.text
                                
#                             if (((re.search("\.$|\.\”$", run.text)) and index == len(paragraph.runs)-1) or (re.search("\!|\?", run.text)) or re.match("\w\)", run.text)) and run.font.highlight_color != WD_COLOR_INDEX.BRIGHT_GREEN :
                            if (((re.search("\.$|\.\”$", word)) and index == len(paragraph.runs)-1) or (re.search("\!|\?", word)) or re.match("\w\)$", word)) and run.font.highlight_color != WD_COLOR_INDEX.BRIGHT_GREEN :
                                # list yang akan dimasukkan ke lexicon
                                listDataLexicon = [lema,sublema,angkaRomawi,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,angkaArab,terjemahan]

                                if checkListNull(listDataLexicon) == False :
                                    # insert lexicon
                                    x = cleanData(listDataLexicon)

                                    if checkListNull(x[0]) == False :
                                        print(x[0])
                                        lexicon  = lexicon.append([x[1]],ignore_index=True)

                                        # clear data
                                        lema = ""
                                        sublema = ""
                                        angkaRomawi = ""  
                                        gabungan = ""
                                        similar = ""
                                        panah = ""
                                        akronim = ""
                                        antonim = ""
                                        perbandingan = ""
                                        keterangan = ""
                                        angkaArab = ""
                                        terjemahan = ""
                                        
                                        isAkronim = False
                                        isSimilar = False
                                        isPanah = False
                                        isPerbandingan = False
                                        isBoldItalic = False
                                        
                    prevTextStyle = "regular"
#                     print("regular :"+run.text)
                
                prevText = run.text
#                 print(prevText)
            
            index += 1

In [ ]:
lexicon.to_excel(doc+'-Scrapped-v2.xlsx', index = False, header=True)